In [1]:
import os
import pycolmap
import numpy as np
import open3d as o3d

# Set the path to the dataset
dataset_path = ".././Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Extract features
sift_options = pycolmap.SiftExtractionOptions()
reader_options = pycolmap.ImageReaderOptions()
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="SIMPLE_RADIAL",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Create options for reconstruction
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4

# Run the reconstruction
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

# Get the first reconstruction (assuming there's at least one)
if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    
    # Print some statistics
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Save reconstruction as .ply file
    ply_path = os.path.join(output_path, "reconstruction.ply")
    points = []
    colors = []

    for point3D_id in reconstruction.points3D:
        point3D = reconstruction.points3D[point3D_id]
        points.append(point3D.xyz)
        colors.append(point3D.color)

    points = np.array(points)
    colors = np.array(colors)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)  # Normalize color values to [0, 1]

    o3d.io.write_point_cloud(ply_path, pcd)
    print(f"Reconstruction saved as PLY file: {ply_path}")

    # Visualize the reconstruction using Open3D
    o3d.visualization.draw_geometries([pcd])
else:
    print("No reconstruction was created.")

W20240821 18:56:10.066692 0x1f5cd0f40 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240821 18:56:10.066901 0x343747000 misc.cc:198] 
Feature extraction
I20240821 18:56:10.067033 0x343e63000 sift.cc:722] Creating SIFT CPU feature extractor
I20240821 18:56:10.067053 0x343f7b000 sift.cc:722] Creating SIFT CPU feature extractor
I20240821 18:56:10.067071 0x344007000 sift.cc:722] Creating SIFT CPU feature extractor
I20240821 18:56:10.067084 0x343eef000 sift.cc:722] Creating SIFT CPU feature extractor
I20240821 18:56:10.067088 0x344093000 sift.cc:722] Creating SIFT CPU feature extractor
I20240821 18:56:10.067112 0x34411f000 sift.cc:

Reconstruction completed and saved to .././Datasets/fountain/reconstruction
Number of registered images: 11
Number of 3D points: 14211
Reconstruction saved as PLY file: .././Datasets/fountain/reconstruction/reconstruction.ply
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


## modifying camera model to OPENCV

In [3]:
from IPython.display import display, HTML
# Create a visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False)
vis.add_geometry(pcd)

# Optionally, you can set some view control parameters
ctr = vis.get_view_control()
ctr.set_zoom(0.8)
ctr.set_front([0, 0, -1])
ctr.set_lookat([0, 0, 0])
ctr.set_up([0, 1, 0])

# Capture the image
vis.poll_events()
vis.update_renderer()
image = vis.capture_screen_float_buffer(False)

# Convert to PIL Image
img = o3d.geometry.Image(image)

# Save the image temporarily
temp_filename = "temp_reconstruction_view.png"
o3d.io.write_image(temp_filename, img)

# Display the image inline
display(HTML(f'<img src="{temp_filename}" width="800">'))

# Close the visualizer
vis.destroy_window()

In [1]:
import os
import pycolmap
import numpy as np
import open3d as o3d

# Set the path to the dataset
dataset_path = "Datasets/fountain"

# Create a COLMAP database
database_path = os.path.join(dataset_path, "database.db")
db = pycolmap.Database(database_path)

# Configure SIFT feature extraction options
sift_options = pycolmap.SiftExtractionOptions()
sift_options.max_num_features = 8192
sift_options.first_octave = -1
sift_options.num_octaves = 4

# Configure image reader options
reader_options = pycolmap.ImageReaderOptions()

# Extract features
pycolmap.extract_features(database_path, dataset_path, image_list=[],
                          camera_mode=pycolmap.CameraMode.AUTO,
                          camera_model="OPENCV",
                          reader_options=reader_options,
                          sift_options=sift_options)

# Match features
pycolmap.match_exhaustive(database_path)

# Configure incremental mapping options
pipeline_options = pycolmap.IncrementalPipelineOptions()
pipeline_options.mapper.num_threads = 4
pipeline_options.mapper.init_min_num_inliers = 100
pipeline_options.mapper.abs_pose_min_num_inliers = 30
pipeline_options.mapper.abs_pose_min_inlier_ratio = 0.25

# Set up output path
output_path = os.path.join(dataset_path, "reconstruction")
os.makedirs(output_path, exist_ok=True)

# Run incremental mapping
reconstructions = pycolmap.incremental_mapping(database_path, dataset_path, output_path, options=pipeline_options)

if reconstructions:
    reconstruction = next(iter(reconstructions.values()))
    print(f"Reconstruction completed and saved to {output_path}")
    print(f"Number of registered images: {reconstruction.num_reg_images()}")
    print(f"Number of 3D points: {reconstruction.num_points3D()}")

    # Prepare point cloud data
    ply_path = os.path.join(output_path, "reconstruction.ply")
    points = []
    colors = []

    for point3D_id in reconstruction.points3D:
        point3D = reconstruction.points3D[point3D_id]
        points.append(point3D.xyz)
        colors.append(point3D.color)

    points = np.array(points)
    colors = np.array(colors)

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

    # Post-processing: downsampling and outlier removal
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Save point cloud as PLY file
    o3d.io.write_point_cloud(ply_path, pcd)
    print(f"Reconstruction saved as PLY file: {ply_path}")

    # Visualize the reconstruction
    o3d.visualization.draw_geometries([pcd])
else:
    print("No reconstruction was created.")

W20240817 19:54:51.452219 0x1f5cd0f40 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240817 19:54:51.452553 0x30a7bb000 misc.cc:198] 
Feature extraction
I20240817 19:54:51.452710 0x336b53000 sift.cc:722] Creating SIFT CPU feature extractor
I20240817 19:54:51.452725 0x336c6b000 sift.cc:722] Creating SIFT CPU feature extractor
I20240817 19:54:51.452722 0x336bdf000 sift.cc:722] Creating SIFT CPU feature extractor
I20240817 19:54:51.452737 0x336cf7000 sift.cc:722] Creating SIFT CPU feature extractor
I20240817 19:54:51.452773 0x336e9b000 sift.cc:722] Creating SIFT CPU feature extractor
I20240817 19:54:51.452789 0x336e0f000 sift.cc:

Reconstruction completed and saved to Datasets/fountain/reconstruction
Number of registered images: 11
Number of 3D points: 14225
Reconstruction saved as PLY file: Datasets/fountain/reconstruction/reconstruction.ply
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 